
# Ejemplo de Blackjack con Q-Learning


Este ejemplo originalmente pertenece a Till Zeemann y está publicado en la [documentación de Gymnasium](https://gymnasium.farama.org/tutorials/training_agents/blackjack_tutorial/), pero fue modificado bajo *fair use* debido a que pertenece a la licensia de MIT para la utilización en el taller.

![CITIC](https://inil.ucr.ac.cr/wp-content/uploads/2023/06/CITIC.jpg)

Modificado para el taller por:
- Luis David Solano Santamaría

## Contexto

<p align="justify">
¡Una vez más vamos a ver las maravillas de Q-Learning, en este caso con este ejemplo de un ambiente de gymnasium que modela Blackjack! Para motivos de este ejemplo, asumo que ya observaron el anterior de Frozen Lake, para que tomen eso en cuenta si no lo han hecho.
</p>

### Problema a resolver



![Agent](https://gymnasium.farama.org/_images/blackjack_AE_loop_dark.png)

<p align="justify">
Observemos que queremos construir un agente que va a interactuar con el ambiente para jugar Blackjack y ganarle a un dealer. Como siempre, tenemos todas las partes clasicas: observaciones, recompensas y acciones.
</p>

![Environment](https://gymnasium.farama.org/_images/blackjack.gif)

<p align="justify">
Para entender entonces este problema debemos comprender lo siguiente:
</p>

- ¿Cómo jugar Blackjack?
- Espacio de acción
- Espacio de observación
- Estado inicial
- Recompensa
- Condición de parada

### ¿Cómo jugar Blackjack?

<p align="justify">El juego comienza con el dealer teniendo una carta boca arriba y una boca abajo, mientras que el jugador tiene dos cartas boca arriba. Todas las cartas se sacan de un mazo infinito (es decir, con reemplazo).</p>

<p align="justify">Los valores de las cartas son los siguientes:</p>

<p align="justify">- Las cartas de figura (J, Q, K) tienen un valor de 10 puntos.</p>
<p align="justify">- Los Ases pueden contar como 11 (llamado ‘As utilizable’) o como 1.</p>
<p align="justify">- Las cartas numéricas (2-10) tienen un valor igual a su número.</p>

<p align="justify">El jugador tiene la suma de las cartas que posee. El jugador puede solicitar cartas adicionales (pedir) hasta que decida detenerse (plantarse) o superar 21 (pasarse, lo que resulta en una pérdida inmediata).</p>

<p align="justify">Después de que el jugador se plante, el crupier revela su carta oculta y saca cartas hasta que su suma sea 17 o mayor. Si el crupier se pasa, el jugador gana.</p>

<p align="justify">Si ni el jugador ni el crupier se pasan, el resultado (ganar, perder, empate) se decide según quién esté más cerca de 21.</p>

<p align="justify">Este entorno corresponde a la versión del problema de blackjack descrita en el Ejemplo 5.1 del libro Reinforcement Learning: An Introduction de Sutton y Barto.</p>


### Espacio de acción

El espacio de acción es simple en naturaleza, podemos parar de tomar cartas o tomar una carta más. Esto lo pasamos a números de la siguiente manera:

0. Parar de tomar cartas (Stick)

1. Tomar una carta más (Hit)

### Espacio de observación

<p align="justify">
La observación consiste en un tupla de tres valores que contiene:
</p>

1. La suma actual del jugador

2. El valor de la carta visible del dealer (1-10 donde 1 es un as)

3. Si el jugador tiene un as utilizable (0 o 1).



### Estado inicial

| Observación              | Valores         |
|--------------------------|-----------------|
| Suma actual del jugador  | 4, 5, …, 21     |
| Valor de la carta visible del dealer | 1, 2, …, 10  |
| As utilizable            | 0, 1            |


### Recompensas



- Ganar el juego: +1

- Perder juego: -1

- Empatar juego: 0

### Condición de parada


Un episodio puede terminar si ocurren los siguientes dos eventos:

1. El jugador excede 21 y pierde.
2. El jugador abandona (sticks).

## Ejecución guiada

A diferencia del ejemplo pasado, se va a adentrar más a fondo que hace cada parte de Gym.

### Configuración del ambiente de Python




In [ ]:
# Author: Till Zemann
# Modifier: Luis Solano
# License: MIT License

from __future__ import annotations

from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib.patches import Patch
from tqdm import tqdm

import gymnasium as gym

Vamos a crear el ambiente de Blackjack, siguiedo la implementación del libro de Sutton & Barto.

In [ ]:
env = gym.make("Blackjack-v1", sab=True)

### Observando el ambiente

Primero, llamamos a ``env.reset()`` para iniciar un episodio. Esta función reinicia el entorno a una posición inicial y devuelve una ``observación`` inicial. Normalmente también establecemos ``done = False``. Esta variable será útil más adelante para verificar si un juego ha terminado (es decir, si el jugador gana o pierde).


In [ ]:
# Reset the environment to get the first observation
done = False
observation, info = env.reset()


Tengamos en cuenta que nuestra observación es una tupla que consiste en 3 valores:

- La suma actual del jugador
- El valor de la carta visible del crupier
- Un valor booleano que indica si el jugador tiene un as utilizable (un as es utilizable si cuenta como 11 sin pasarse)


### Ejecutando una acción





Después de recibir nuestra primera observación, solo vamos a usar la función ``env.step(action)`` para interactuar con el entorno. Esta función toma una acción como entrada y la ejecuta en el entorno. Debido a que esa acción cambia el estado del entorno, nos devuelve cuatro variables útiles. Estas son:

- ``next_state``: Esta es la observación que el agente recibirá después de realizar la acción.
- ``reward``: Esta es la recompensa que el agente recibirá después de realizar la acción.
- ``terminated``: Esta es una variable booleana que indica si el entorno ha terminado o no.
- ``truncated``: Esta es una variable booleana que también indica si el episodio terminó por truncamiento anticipado, es decir, si se alcanzó un límite de tiempo.
- ``info``: Este es un diccionario que puede contener información adicional sobre el entorno.

Hay que tomar en cuenta que no es una buena idea llamar a ``env.render()`` en el ciclo de entrenamiento porque renderizar ralentiza mucho el entrenamiento. En su lugar, se puede construir un ciclo adicional para evaluar y mostrar el agente después del entrenamiento, como se hizo previamente.


In [ ]:
# sample a random action from all valid actions
action = env.action_space.sample()
# action=1

# execute the action in our environment and receive infos from the environment
observation, reward, terminated, truncated, info = env.step(action)

# observation=(24, 10, False)
# reward=-1.0
# terminated=True
# truncated=False
# info={}

Una vez que ``terminated = True`` o ``truncated=True``, debemos detener el
episodio actual y comenzar uno nuevo con ``env.reset()``. Si sigues
ejecutando acciones sin reiniciar el entorno, este seguirá respondiendo,
pero la salida no será útil para el entrenamiento (incluso podría ser
perjudicial si el agente aprende con datos inválidos).


### Construyendo una clase agente



A cambio del ejemplo pasado, aquí se construye una clase agente, en caso de no conocer programación orientada a objetos pueden preguntarme (si están presencialmente en el laboratorio) y puedo ayudarlos.

¡Construyamos un ``agente de Q-learning`` para resolver *Blackjack-v1*! Necesitaremos
algunas funciones para seleccionar una acción y actualizar los valores de acción del agente.
Para asegurarnos de que el agente explore el entorno, una posible
solución es la estrategia ``epsilon-greedy``, donde seleccionamos una acción aleatoria con el porcentaje ``epsilon`` y la acción greedy (actualmente valorada como la mejor) con ``1 - epsilon``.

Este es el mismo algoritmo que utilizamos la vez pasada, pueden profundizar más por su cuenta en este [enlace](https://www.google.com/search?client=firefox-b-d&q=epsilon+greedy+selection).





In [ ]:
class BlackjackAgent:
    def __init__(
        self,
        env,
        learning_rate: float,
        initial_epsilon: float,
        epsilon_decay: float,
        final_epsilon: float,
        discount_factor: float = 0.95,
    ):
        """Initialize a Reinforcement Learning agent with an empty dictionary
        of state-action values (q_values), a learning rate and an epsilon.

        Args:
            learning_rate: The learning rate
            initial_epsilon: The initial epsilon value
            epsilon_decay: The decay for epsilon
            final_epsilon: The final epsilon value
            discount_factor: The discount factor for computing the Q-value
        """
        self.q_values = defaultdict(lambda: np.zeros(env.action_space.n))

        self.lr = learning_rate
        self.discount_factor = discount_factor

        self.epsilon = initial_epsilon
        self.epsilon_decay = epsilon_decay
        self.final_epsilon = final_epsilon

        self.training_error = []

    def get_action(self, env, obs: tuple[int, int, bool]) -> int:
        """
        Returns the best action with probability (1 - epsilon)
        otherwise a random action with probability epsilon to ensure exploration.
        """
        # with probability epsilon return a random action to explore the environment
        if np.random.random() < self.epsilon:
            return env.action_space.sample()

        # with probability (1 - epsilon) act greedily (exploit)
        else:
            return int(np.argmax(self.q_values[obs]))

    def update(
        self,
        obs: tuple[int, int, bool],
        action: int,
        reward: float,
        terminated: bool,
        next_obs: tuple[int, int, bool],
    ):
        """Updates the Q-value of an action."""
        future_q_value = (not terminated) * np.max(self.q_values[next_obs])
        temporal_difference = (
            reward + self.discount_factor * future_q_value - self.q_values[obs][action]
        )

        self.q_values[obs][action] = (
            self.q_values[obs][action] + self.lr * temporal_difference
        )
        self.training_error.append(temporal_difference)

    def decay_epsilon(self):
        self.epsilon = max(self.final_epsilon, self.epsilon - self.epsilon_decay)

Para entrenar al agente, dejaremos que el agente juegue un episodio (un juego completo se llama episodio) a la vez y luego actualizaremos sus valores Q después de cada paso (una acción única en un juego se llama paso).

El agente tendrá que experimentar muchos episodios para explorar el
entorno de manera suficiente.

Ahora deberíamos estar listos para construir el ciclo de entrenamiento.


In [ ]:
# hyperparameters
learning_rate = 0.01
n_episodes = 100_000
start_epsilon = 1.0
epsilon_decay = start_epsilon / (n_episodes / 2)  # reduce the exploration over time
final_epsilon = 0.1

agent = BlackjackAgent(
    env=env,
    learning_rate=learning_rate,
    initial_epsilon=start_epsilon,
    epsilon_decay=epsilon_decay,
    final_epsilon=final_epsilon,
)

¡Genial, entrenemos!

Información: Los hiperparámetros actuales están configurados para entrenar rápidamente un agente decente.

Si se desea converger a la política óptima, se puede aumentar
el número de episodios (n_episodes) por 10 veces y reducir la tasa de aprendizaje (por ejemplo, a 0.001).

Experimenten :)


In [ ]:
env = gym.wrappers.RecordEpisodeStatistics(env)
for episode in tqdm(range(n_episodes)):
    obs, info = env.reset()
    done = False

    # play one episode
    while not done:
        action = agent.get_action(env, obs)
        next_obs, reward, terminated, truncated, info = env.step(action)

        # update the agent
        agent.update(obs, action, reward, terminated, next_obs)

        # update if the environment is done and the current obs
        done = terminated or truncated
        obs = next_obs

    agent.decay_epsilon()


100%|██████████| 100000/100000 [00:18<00:00, 5345.54it/s]


## Juego de resultado

En este caso, el agente no tiene soporte para video entonces una interfaz de texto mostrará los resultados.

In [ ]:
def play_blackjack(agent, env):
    """
    Plays a single game of Blackjack with the given agent.
    The agent decides actions until the game ends, and the outcome is printed.
    """
    state, _ = env.reset()
    done = False

    print("\n--- Starting a new Blackjack game ---")
    print(f"Initial state (Player Total, Dealer Card, Usable Ace): {state}\n")

    while not done:
        action = agent.get_action(env, state)
        next_state, reward, done, _, _ = env.step(action)

        action_str = "Hit" if action == 1 else "Stick"
        print(f"Agent chose {action_str} -> New state: {next_state}")

        agent.update(state, action, reward, done, next_state)
        state = next_state

    print("\n--- Game Over ---")
    if reward > 0:
        print("Agent **WON** the game!")
    elif reward < 0:
        print("Agent **LOST** the game.")
    else:
        print("It's a **TIE**.")

    agent.decay_epsilon()

In [ ]:
play_blackjack(agent, env)



--- Starting a new Blackjack game ---
Initial state (Player Total, Dealer Card, Usable Ace): (15, 1, 0)

Agent chose Stick -> New state: (15, 1, 0)

--- Game Over ---
Agent **LOST** the game.
